<a href="https://colab.research.google.com/github/sathu0622/25-26J-438-AI-Powered-LMS-for-Visually-Impaired-Students/blob/Audio-Based-Learning-Module-Tts-With-Emotional-Tone%26Nlp-Simplification/AI_History_Teacher_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
!mkdir -p /content/project
!mkdir -p /content/project/sounds
!mkdir -p /content/project/audio_output
!mkdir -p /content/project/models

In [55]:
# Install TTS and audio processing libraries
!pip install gTTS playsound pydub pandas numpy
!pip install torch torchaudio
!pip install transformers sentencepiece
!pip install ipywidgets
!pip install streamlit  # For optional web interface
!pip install coqui-tts  # Advanced TTS with emotion

# For emotion detection in text
!pip install text2emotion

# For audio mixing
!pip install soundfile librosa

In [56]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

# Load your dataset
df = pd.read_csv('/content/drive/MyDrive/History_Project/grade10_dataset.csv', encoding='latin-1')

# Explore data structure
print("Dataset columns:", df.columns.tolist())
print("\nTotal lessons:", len(df))
print("\nChapters:", df['chapter'].unique())

# Clean and structure data
def preprocess_data(df):
    # Split sound effects into list
    df['sound_effects_list'] = df['sound_effects'].str.split(', ')

    # Create chapter-lesson mapping
    df['chapter_num'] = df['chapter'].str.extract(r'(\d+)\.')
    df['chapter_title'] = df['chapter'].str.split('.').str[1]

    return df

df = preprocess_data(df)
df.head()

Dataset columns: ['chapter', 'Grade/Topic', 'original_text', 'simplified_text', 'narrative_text', 'emotion', 'sound_effects']

Total lessons: 35

Chapters: ['1.Sources of Studying History' '2. Ancient Settlements'
 '3. Evolution of Political Power in Sri Lanka.'
 '4. The Ancient Society of Sri Lanka'
 '5.The Ancient Science and Technology in Sri Lanka'
 '6.Historical Knowledge and Its Practical Application'
 '7. Decline of Ancient Cities in the Dry Zone and Origin of New Kingdoms in South West'
 '8.Kandyan Kingdom' '9. Renaissance'
 '10. Sri Lanka and the Western World']


,chapter,Grade/Topic,original_text,simplified_text,narrative_text,emotion,sound_effects,sound_effects_list,chapter_num,chapter_title
0,1.Sources of Studying History,Grade 10: Classification of Sources,History is a subject which studies the past hu...,History studies past human actions. Sources ar...,"Dive into the tapestry of Sri Lanka's past, wh...",neutral,"soft_background_music, distant_digging, gentle...","[soft_background_music, distant_digging, gentl...",1,Sources of Studying History
1,1.Sources of Studying History,Grade 10: Importance of Learning History,History is a subject which is linked with the ...,History connects to studying sources and can b...,Imagine unlocking the secrets of the past thro...,inspirational,"soft_background_music, thoughtful_chime, gentl...","[soft_background_music, thoughtful_chime, gent...",1,Sources of Studying History
2,1.Sources of Studying History,Grade 10: Protecting Archaeological Sources,When we consider the long history of Sri Lanka...,Sri Lanka's long history and large ancient pop...,"In the emerald isle of Sri Lanka, echoes of a ...",inspirational,"soft_background_music, distant_echoes, crumbli...","[soft_background_music, distant_echoes, crumbl...",1,Sources of Studying History
3,2. Ancient Settlements,Grade 10: Ancient Settlements - Settlements in...,The period which was before the past that is d...,The time before written history is called the ...,"Journey back to Sri Lanka's ancient dawn, befo...",neutral,"soft_background_music, gentle_wind, stone_chip...","[soft_background_music, gentle_wind, stone_chi...",2,Ancient Settlements
4,2. Ancient Settlements,Grade 10: Ancient Settlements - Settlements in...,The period between the end of the prehistoric ...,The time between prehistoric and historic eras...,"As the prehistoric shadows faded, Sri Lanka en...",neutral,"soft_background_music, distant_forging, gentle...","[soft_background_music, distant_forging, gentl...",2,Ancient Settlements


import ipywidgets as widgets
from IPython.display import display, Audio, clear_output

# Ensure df has the necessary columns by re-applying preprocessing if needed
def preprocess_data(df):
    # Split sound effects into list
    if 'sound_effects' in df.columns and 'sound_effects_list' not in df.columns:
        df['sound_effects_list'] = df['sound_effects'].str.split(', ')
    # Create chapter-lesson mapping
    if 'chapter' in df.columns and 'chapter_num' not in df.columns:
        df['chapter_num'] = df['chapter'].str.extract(r'(\d+)\.')
    if 'chapter' in df.columns and 'chapter_title' not in df.columns:
        df['chapter_title'] = df['chapter'].str.split('.').str[1]
    return df

# Re-apply preprocessing to df (assuming df is already loaded)
df = preprocess_data(df)

# Chapter selection dropdown
chapter_options = [f"{row['chapter_num']}. {row['chapter_title']}"
                   for idx, row in df[['chapter_num', 'chapter_title']].drop_duplicates().iterrows()]

chapter_dropdown = widgets.Dropdown(
    options=chapter_options,
    description='Chapter:',
    disabled=False,
)

# Lesson selection (will update based on chapter)
lesson_dropdown = widgets.Dropdown(
    options=[],
    description='Lesson:',
    disabled=False,
)

# Emotion adjustment slider
emotion_slider = widgets.FloatSlider(
    value=1.0,
    min=0.5,
    max=2.0,
    step=0.1,
    description='Emotion Intensity:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

# Sound effects toggle
sound_toggle = widgets.ToggleButtons(
    options=['With Effects'],
    description='Sound:',
    disabled=False,
    button_style='',
    tooltips=['Play with sound effects']
)

# Play button
play_button = widgets.Button(
    description='🎵 Play Lesson',
    disabled=False,
    button_style='success',
    tooltip='Play the selected lesson',
    icon='play'
)

# Display widgets
display(widgets.VBox([chapter_dropdown, lesson_dropdown, emotion_slider, sound_toggle, play_button]))

In [58]:
from gtts import gTTS
import tempfile
from pydub import AudioSegment
from pydub.playback import play
import time

class EmotionalTTS:
    def __init__(self):
        self.emotion_map = {
            'neutral': {'speed': 1.0, 'pitch': 1.0, 'volume': 1.0},
            'inspirational': {'speed': 0.9, 'pitch': 1.1, 'volume': 1.2},
            'awe': {'speed': 0.85, 'pitch': 1.15, 'volume': 1.3},
            'vibrancy': {'speed': 1.1, 'pitch': 1.05, 'volume': 1.1},
            'harmony': {'speed': 1.0, 'pitch': 1.0, 'volume': 1.0},
            'wonder': {'speed': 0.95, 'pitch': 1.1, 'volume': 1.1},
            'reverence': {'speed': 0.9, 'pitch': 0.95, 'volume': 1.0},
            'justice': {'speed': 1.0, 'pitch': 1.0, 'volume': 1.2},
            'prosperity': {'speed': 1.05, 'pitch': 1.05, 'volume': 1.1},
            'warmth': {'speed': 0.95, 'pitch': 1.0, 'volume': 1.0},
            'hope': {'speed': 1.0, 'pitch': 1.1, 'volume': 1.1},
            'resilience': {'speed': 1.0, 'pitch': 1.0, 'volume': 1.2},
            'somber': {'speed': 0.85, 'pitch': 0.9, 'volume': 0.9},
            'respect': {'speed': 0.95, 'pitch': 1.0, 'volume': 1.0}
        }

    def generate_speech(self, text, emotion='neutral', intensity=1.0, lang='en'):
        """Generate TTS with emotional modulation"""

        # Adjust parameters based on emotion
        params = self.emotion_map.get(emotion, self.emotion_map['neutral'])
        speed_adjusted = params['speed'] * (2 - intensity)  # Inverse for speed
        pitch_note = params['pitch'] * intensity

        # Generate TTS
        tts = gTTS(text=text, lang=lang, slow=False)

        # Save to temporary file
        with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as f:
            temp_path = f.name
            tts.save(temp_path)

        # Load and adjust audio parameters
        audio = AudioSegment.from_mp3(temp_path)

        # Adjust speed (through frame rate manipulation)
        new_frame_rate = int(audio.frame_rate * speed_adjusted)
        audio = audio._spawn(audio.raw_data, overrides={
            "frame_rate": new_frame_rate
        })

        # Adjust pitch (simplified through speed change)
        # Note: For better pitch control, consider using librosa
        audio = audio.set_frame_rate(int(audio.frame_rate * pitch_note))

        # Adjust volume
        volume_change = (params['volume'] * intensity - 1) * 10  # Convert to dB
        audio = audio + volume_change

        # Export final audio
        output_path = temp_path.replace('.mp3', '_adjusted.mp3')
        audio.export(output_path, format='mp3')

        return output_path

# Initialize TTS system
tts_engine = EmotionalTTS()

In [59]:
# Voice gender selection
voice_gender = widgets.RadioButtons(
    options=['👨 Male Teacher', '👩 Female Teacher'],
    value='👨 Male Teacher',
    description='Voice:',
    disabled=False,
    layout=widgets.Layout(width='300px')
)

# Or use a toggle for simplicity:
voice_toggle = widgets.ToggleButtons(
    options=['👨 Male', '👩 Female'],
    value='👨 Male',
    description='Teacher Voice:',
    disabled=False,
    button_style='',
    tooltips=['Male history teacher voice', 'Female history teacher voice']
)

In [60]:
class SoundEffectsMixer:
    def __init__(self, sounds_folder):
        self.sounds_folder = sounds_folder
        self.load_sound_effects()

    def load_sound_effects(self):
        """Load all available sound effects"""
        self.sound_effects = {}
        sound_files = os.listdir(self.sounds_folder)

        for sound_file in sound_files:
            if sound_file.endswith(('.mp3', '.wav', '.ogg')):
                name = os.path.splitext(sound_file)[0]
                path = os.path.join(self.sounds_folder, sound_file)
                try:
                    self.sound_effects[name] = AudioSegment.from_file(path)
                    print(f"Loaded: {name}")
                except:
                    print(f"Failed to load: {sound_file}")

    def mix_audio(self, narration_path, effects_list, volume_ratio=0.3):
        """Mix narration with sound effects"""

        # Load narration
        narration = AudioSegment.from_file(narration_path)

        # Create background track
        background = AudioSegment.silent(duration=len(narration))

        # Mix each sound effect
        for effect_name in effects_list:
            if effect_name in self.sound_effects:
                effect = self.sound_effects[effect_name]

                # Loop effect if shorter than narration
                while len(effect) < len(narration):
                    effect = effect + effect

                # Trim to narration length
                effect = effect[:len(narration)]

                # Adjust volume
                effect = effect - (20 - (20 * volume_ratio))  # Reduce volume

                # Overlay on background
                background = background.overlay(effect)

        # Mix narration with background
        mixed = narration.overlay(background)

        # Save mixed audio
        output_path = narration_path.replace('.mp3', '_mixed.mp3')
        mixed.export(output_path, format='mp3')

        return output_path

# Initialize sound mixer
sound_mixer = SoundEffectsMixer('/content/drive/MyDrive/History_Project/sounds/')

Loaded: distant_digging
Loaded: gentle_wind
Loaded: chime
Loaded: thoughtful_chime
Loaded: soft_background_music
Loaded: crumbling_stone
Loaded: stone_chipping
Loaded: animal_calls
Loaded: distant_forging
Loaded: pottery_spinning
Loaded: flowing_water
Loaded: distant_hammering
Loaded: distant_drums
Loaded: majestic_horn
Loaded: temple_bells
Loaded: soft_chant
Loaded: distant_horses
Loaded: rustling_leaves
Loaded: market_chatter
Loaded: hammer_on_anvil
Loaded: distant_flute
Loaded: crackling_fire
Loaded: potter_wheel_spin
Loaded: stone_masonry
Loaded: fountain_spray
Loaded: thunder_rumble
Loaded: distant_cow_bells
Loaded: flickering_flame
Loaded: measuring_tap
Loaded: gavel_strike
Loaded: clinking_coins
Loaded: ocean_waves
Loaded: ship_bell
Loaded: soft_female_chant
Loaded: jewelry_clink
Loaded: bubbling_pot
Loaded: birdsong
Loaded: distant_birdsong
Loaded: wind_through_ruins
Loaded: fading_temple_bells
Loaded: mournful_flute
Loaded: triumphant_horns
Loaded: wood_carving
Loaded: royal_d

In [61]:
# 1. Emotion detection from text (fallback)
import text2emotion as te

def detect_emotion_from_text(text):
    """Detect emotion if not specified in dataset"""
    emotions = te.get_emotion(text)
    primary_emotion = max(emotions.items(), key=lambda x: x[1])[0]

    emotion_mapping = {
        'Happy': 'inspirational',
        'Angry': 'neutral',  # Map to neutral for educational content
        'Surprise': 'wonder',
        'Sad': 'somber',
        'Fear': 'neutral'
    }

    return emotion_mapping.get(primary_emotion, 'neutral')

# 2. Batch generate all lessons
def generate_all_lessons():
    """Pre-generate audio for all lessons"""
    for idx, row in df.iterrows():
        print(f"Generating: {row['chapter']} - {row['Grade/Topic']}")

        # Generate narration
        narration_path = tts_engine.generate_speech(
            text=row['simplified_text'][:1000],  # First 1000 chars
            emotion=row['emotion'],
            intensity=1.0
        )

        # Mix with sound effects
        if row['sound_effects_list']:
            final_path = sound_mixer.mix_audio(
                narration_path,
                row['sound_effects_list']
            )

        # Save to organized folder
        chapter_folder = f"chapter_{row['chapter_num']}"
        os.makedirs(f"/content/project/audio_output/{chapter_folder}", exist_ok=True)

        lesson_name = row['Grade/Topic'].replace(':', '_').replace(' ', '_')
        final_name = f"/content/project/audio_output/{chapter_folder}/{lesson_name}.mp3"

        # Copy final file
        !cp "{final_path}" "{final_name}"

        print(f"  ✓ Saved to: {final_name}")

# 3. Quiz generation from text
from transformers import pipeline

qa_pipeline = pipeline("question-answering",
                      model="distilbert-base-cased-distilled-squad")

def generate_quiz(text, num_questions=3):
    """Generate simple quiz questions from text"""
    sentences = text.split('.')
    questions = []

    for i, sentence in enumerate(sentences[:num_questions*2]):
        if len(sentence.strip()) > 20:  # Substantial sentences only
            # Simple question generation (replace with better model)
            words = sentence.split()
            if len(words) > 5:
                # Create fill-in-the-blank
                blank_word = words[-2] if len(words) > 7 else words[3]
                question = sentence.replace(blank_word, "______")
                answer = blank_word

                questions.append({
                    'question': f"Complete: {question}",
                    'answer': answer,
                    'type': 'fill_blank'
                })

    return questions[:num_questions]

Device set to use cpu


In [62]:
import ipywidgets as widgets
from IPython.display import display, Audio, clear_output

# Ensure df has the necessary columns by re-applying preprocessing if needed
def preprocess_data(df):
    # Split sound effects into list
    if 'sound_effects' in df.columns and 'sound_effects_list' not in df.columns:
        df['sound_effects_list'] = df['sound_effects'].str.split(', ')
    # Create chapter-lesson mapping
    if 'chapter' in df.columns and 'chapter_num' not in df.columns:
        df['chapter_num'] = df['chapter'].str.extract(r'(\d+)\.')
    if 'chapter' in df.columns and 'chapter_title' not in df.columns:
        df['chapter_title'] = df['chapter'].str.split('.').str[1]
    return df

# Re-apply preprocessing to df (assuming df is already loaded)
df = preprocess_data(df)

# Grade selection widget
grade_selector = widgets.RadioButtons(
    options=['Grade 10', 'Grade 11'],
    value='Grade 10',
    description='Grade:',
    disabled=False,
    layout=widgets.Layout(width='auto')
)

# Chapter selection dropdown
chapter_options = [f"{row['chapter_num']}. {row['chapter_title']}"
                   for idx, row in df[['chapter_num', 'chapter_title']].drop_duplicates().iterrows()]

chapter_dropdown = widgets.Dropdown(
    options=chapter_options,
    description='Chapter:',
    disabled=False,
)

# Lesson selection (will update based on chapter)
lesson_dropdown = widgets.Dropdown(
    options=[],
    description='Lesson:',
    disabled=False,
)

# Emotion adjustment slider
emotion_slider = widgets.FloatSlider(
    value=1.0,
    min=0.5,
    max=2.0,
    step=0.1,
    description='Emotion Intensity:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

# Sound effects toggle
sound_toggle = widgets.ToggleButtons(
    options=['With Effects', 'Only Effects', 'Without Effects'],
    description='Sound:',
    disabled=False,
    button_style='',
    tooltips=['Play with sound effects', 'Play only sound effects', 'Play narration only']
)

# Play button
play_button = widgets.Button(
    description='🎵 Play Lesson',
    disabled=False,
    button_style='success',
    tooltip='Play the selected lesson',
    icon='play'
)

# Display widgets
display(widgets.VBox([grade_selector, chapter_dropdown, lesson_dropdown, emotion_slider, sound_toggle, play_button]))


📚 Chapter: 1.Industrial Revolution
🎯 Topic: Grade 11: Expansion to Other Countries and Results
😊 Emotion: mixed_wonder_awe
🔊 Sound Effects: steam_train_whistle, factory_hammer, bustling_city_crowd, protest_chant, polluted_river_flow, gentle_wind, chime

🔊 Generating introductory speech part 1...
🎵 Integrating specific sound effect (chime)...
🔊 Generating introductory speech part 2...

🔊 Generating main lesson speech...

🔊 Generating closing speech...
🎵 Playing with sound effects (spaced out)...



📖 Lesson Text Preview:
 Spread: Britain ? Europe (Germany/France/Belgium/Austria/Prussia railways/roads), USA (post-independence), Japan (late 19thearly 20th rapid), Asia (China/India/Sri Lanka 20th century) ? global village via transport/communication. Results: Political: Capitalism (laissez-faire Adam Smith), socialism...


**Reasoning**:
To ensure that `tts_engine` is properly initialized and available for use by the `play_selected_lesson` function, the code cell defining `EmotionalTTS` needs to be executed before the interaction logic is set up. This will resolve the `NameError` if `tts_engine` was not yet defined when the widgets were created or actions were attempted.



In [63]:
from gtts import gTTS
import tempfile
from pydub import AudioSegment
from pydub.playback import play
import time

class EmotionalTTS:
    def __init__(self):
        self.emotion_map = {
            'neutral': {'speed': 1.0, 'pitch': 1.0, 'volume': 1.0},
            'inspirational': {'speed': 0.9, 'pitch': 1.1, 'volume': 1.2},
            'awe': {'speed': 0.85, 'pitch': 1.15, 'volume': 1.3},
            'vibrancy': {'speed': 1.1, 'pitch': 1.05, 'volume': 1.1},
            'harmony': {'speed': 1.0, 'pitch': 1.0, 'volume': 1.0},
            'wonder': {'speed': 0.95, 'pitch': 1.1, 'volume': 1.1},
            'reverence': {'speed': 0.9, 'pitch': 0.95, 'volume': 1.0},
            'justice': {'speed': 1.0, 'pitch': 1.0, 'volume': 1.2},
            'prosperity': {'speed': 1.05, 'pitch': 1.05, 'volume': 1.1},
            'warmth': {'speed': 0.95, 'pitch': 1.0, 'volume': 1.0},
            'hope': {'speed': 1.0, 'pitch': 1.1, 'volume': 1.1},
            'resilience': {'speed': 1.0, 'pitch': 1.0, 'volume': 1.2},
            'somber': {'speed': 0.85, 'pitch': 0.9, 'volume': 0.9},
            'respect': {'speed': 0.95, 'pitch': 1.0, 'volume': 1.0}
        }

    def generate_speech(self, text, emotion='neutral', intensity=1.0, lang='en'):
        """Generate TTS with emotional modulation"""

        # Adjust parameters based on emotion
        params = self.emotion_map.get(emotion, self.emotion_map['neutral'])
        speed_adjusted = params['speed'] * (2 - intensity)  # Inverse for speed
        pitch_note = params['pitch'] * intensity

        # Generate TTS
        tts = gTTS(text=text, lang=lang, slow=False)

        # Save to temporary file
        with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as f:
            temp_path = f.name
            tts.save(temp_path)

        # Load and adjust audio parameters
        audio = AudioSegment.from_mp3(temp_path)

        # Adjust speed (through frame rate manipulation)
        new_frame_rate = int(audio.frame_rate * speed_adjusted)
        audio = audio._spawn(audio.raw_data, overrides={
            "frame_rate": new_frame_rate
        })

        # Adjust pitch (simplified through speed change)
        # Note: For better pitch control, consider using librosa
        audio = audio.set_frame_rate(int(audio.frame_rate * pitch_note))

        # Adjust volume
        volume_change = (params['volume'] * intensity - 1) * 10  # Convert to dB
        audio = audio + volume_change

        # Export final audio
        output_path = temp_path.replace('.mp3', '_adjusted.mp3')
        audio.export(output_path, format='mp3')

        return output_path

# Initialize TTS system
tts_engine = EmotionalTTS()

**Reasoning**:
To ensure that `sound_mixer` is properly initialized and available for use by the `play_selected_lesson` function, the code cell defining `SoundEffectsMixer` needs to be executed next. This will resolve the `NameError` for `sound_mixer`.



In [64]:
class SoundEffectsMixer:
    def __init__(self, sounds_folder):
        self.sounds_folder = sounds_folder
        self.load_sound_effects()

    def load_sound_effects(self):
        """Load all available sound effects"""
        self.sound_effects = {}
        sound_files = os.listdir(self.sounds_folder)

        for sound_file in sound_files:
            if sound_file.endswith(('.mp3', '.wav', '.ogg')):
                name = os.path.splitext(sound_file)[0]
                path = os.path.join(self.sounds_folder, sound_file)
                try:
                    self.sound_effects[name] = AudioSegment.from_file(path)
                    print(f"Loaded: {name}")
                except:
                    print(f"Failed to load: {sound_file}")

    def mix_audio(self, narration_path, effects_list, volume_ratio=0.3):
        """Mix narration with sound effects"""

        # Load narration
        narration = AudioSegment.from_file(narration_path)

        # Create background track
        background = AudioSegment.silent(duration=len(narration))

        # Mix each sound effect
        for effect_name in effects_list:
            if effect_name in self.sound_effects:
                effect = self.sound_effects[effect_name]

                # Loop effect if shorter than narration
                while len(effect) < len(narration):
                    effect = effect + effect

                # Trim to narration length
                effect = effect[:len(narration)]

                # Adjust volume
                effect = effect - (20 - (20 * volume_ratio))  # Reduce volume

                # Overlay on background
                background = background.overlay(effect)

        # Mix narration with background
        mixed = narration.overlay(background)

        # Save mixed audio
        output_path = narration_path.replace('.mp3', '_mixed.mp3')
        mixed.export(output_path, format='mp3')

        return output_path

# Initialize sound mixer
sound_mixer = SoundEffectsMixer('/content/drive/MyDrive/History_Project/sounds/')


Loaded: distant_digging
Loaded: gentle_wind
Loaded: chime
Loaded: thoughtful_chime
Loaded: soft_background_music
Loaded: crumbling_stone
Loaded: stone_chipping
Loaded: animal_calls
Loaded: distant_forging
Loaded: pottery_spinning
Loaded: flowing_water
Loaded: distant_hammering
Loaded: distant_drums
Loaded: majestic_horn
Loaded: temple_bells
Loaded: soft_chant
Loaded: distant_horses
Loaded: rustling_leaves
Loaded: market_chatter
Loaded: hammer_on_anvil
Loaded: distant_flute
Loaded: crackling_fire
Loaded: potter_wheel_spin
Loaded: stone_masonry
Loaded: fountain_spray
Loaded: thunder_rumble
Loaded: distant_cow_bells
Loaded: flickering_flame
Loaded: measuring_tap
Loaded: gavel_strike
Loaded: clinking_coins
Loaded: ocean_waves
Loaded: ship_bell
Loaded: soft_female_chant
Loaded: jewelry_clink
Loaded: bubbling_pot
Loaded: birdsong
Loaded: distant_birdsong
Loaded: wind_through_ruins
Loaded: fading_temple_bells
Loaded: mournful_flute
Loaded: triumphant_horns
Loaded: wood_carving
Loaded: royal_d

In [65]:
def load_and_preprocess_dataset(grade_level):
    """Loads and preprocesses the dataset for the given grade level."""
    global df # Declare df as global to modify it
    filename = 'grade10_dataset.csv' if grade_level == 'Grade 10' else 'grade11_dataset.csv'
    file_path = f'/content/drive/MyDrive/History_Project/{filename}'

    # Load the dataset
    df = pd.read_csv(file_path, encoding='latin-1')

    # Preprocess the data
    df = preprocess_data(df)
    return df

def update_lessons(change):
    """Update lesson dropdown based on selected chapter"""
    chapter_num = change['new'].split('.')[0]
    lessons = df[df['chapter_num'] == chapter_num]['Grade/Topic'].tolist()
    lesson_dropdown.options = lessons

def on_grade_change(change):
    """Callback function to update data and dropdowns based on grade selection."""
    global df
    selected_grade = change['new']
    print(f"Loading data for {selected_grade}...")

    # Load and preprocess the new dataset
    df = load_and_preprocess_dataset(selected_grade)

    # Update chapter dropdown options
    chapter_options = [f"{row['chapter_num']}. {row['chapter_title']}"
                       for idx, row in df[['chapter_num', 'chapter_title']].drop_duplicates().iterrows()]
    chapter_dropdown.options = chapter_options

    # Select the first chapter by default and trigger lesson update
    if chapter_options:
        chapter_dropdown.value = chapter_options[0]
        update_lessons({'new': chapter_options[0]}) # Manually trigger lesson update

def play_selected_lesson(b):
    """Main function to play selected lesson"""

    # Clear previous output
    clear_output(wait=True)
    display(widgets.VBox([grade_selector, chapter_dropdown, lesson_dropdown,
                         emotion_slider, sound_toggle, play_button]))

    # Get selected data
    chapter_num = chapter_dropdown.value.split('.')[0]
    lesson_topic = lesson_dropdown.value

    # Find the lesson data
    lesson_data = df[(df['chapter_num'] == chapter_num) &
                     (df['Grade/Topic'] == lesson_topic)].iloc[0]

    # Print chapter and topic first as requested
    print(f"\n📚 Chapter: {lesson_data['chapter']}")
    print(f"🎯 Topic: {lesson_data['Grade/Topic']}")
    print(f"😊 Emotion: {lesson_data['emotion']}")
    print(f"🔊 Sound Effects: {lesson_data['sound_effects']}")

    # Use simplified text for TTS
    text_to_speak = lesson_data['simplified_text']

    # Replace 'e.g.' with 'example is' for better narration
    text_to_speak = text_to_speak.replace('e.g.', 'example is')

    # --- Modify Introduction Structure and Integrate Specific Sound Effect ---
    chapter_title_clean = lesson_data['chapter_title'].strip()
    topic_clean = lesson_data['Grade/Topic'].split(':')[-1].strip()

    intro_part1_text = f"Hi, you Selected Chapter is {chapter_title_clean} and you choose topic is {topic_clean}. "
    intro_part2_text = f"Today, I am going to discuss about {topic_clean}. "

    print("\n🔊 Generating introductory speech part 1...")
    intro_part1_audio_path = tts_engine.generate_speech(
        text=intro_part1_text,
        emotion=lesson_data['emotion'],
        intensity=emotion_slider.value
    )
    intro_part1_audio = AudioSegment.from_mp3(intro_part1_audio_path)

    print("🎵 Integrating specific sound effect (chime)...")
    # Create a mutable copy of the sound effects list
    current_effects_list = list(lesson_data['sound_effects_list'])

    chime_mid_intro_audio = AudioSegment.silent(duration=500) # Default silent if no chime

    if sound_toggle.value in ['With Effects', 'Only Effects'] and 'chime' in current_effects_list:
        if 'chime' in sound_mixer.sound_effects:
            chime_effect = sound_mixer.sound_effects['chime']
            chime_mid_intro_audio = chime_effect - 15 # Adjust volume
            current_effects_list.remove('chime') # Remove from list so it's not re-mixed later

    print("🔊 Generating introductory speech part 2...")
    intro_part2_audio_path = tts_engine.generate_speech(
        text=intro_part2_text,
        emotion=lesson_data['emotion'],
        intensity=emotion_slider.value
    )
    intro_part2_audio = AudioSegment.from_mp3(intro_part2_audio_path)

    # Concatenate the three intro parts
    full_intro_narration_audio = intro_part1_audio + chime_mid_intro_audio + intro_part2_audio

    final_intro_audio = full_intro_narration_audio

    # Handle soft_background_music specifically for the entire intro
    if sound_toggle.value in ['With Effects', 'Only Effects'] and 'soft_background_music' in current_effects_list:
        if 'soft_background_music' in sound_mixer.sound_effects:
            bgm_effect = sound_mixer.sound_effects['soft_background_music']

            # Loop BGM if shorter than intro
            while len(bgm_effect) < len(full_intro_narration_audio):
                bgm_effect += bgm_effect

            bgm_effect = bgm_effect[:len(full_intro_narration_audio)]
            # Adjust volume for background music, make it very subtle (-20dB)
            bgm_effect = bgm_effect - 20

            final_intro_audio = final_intro_audio.overlay(bgm_effect)
            current_effects_list.remove('soft_background_music') # Remove from list so it's not re-mixed later
    # --- End Introduction Modification ---

    # Generate main lesson speech
    print("\n🔊 Generating main lesson speech...")
    narration_path = tts_engine.generate_speech(
        text=text_to_speak,  # Use full text for better effect placement
        emotion=lesson_data['emotion'],
        intensity=emotion_slider.value
    )
    main_narration_audio = AudioSegment.from_mp3(narration_path)

    # Concatenate intro and main narration
    combined_narration_audio = final_intro_audio + main_narration_audio

    # Add closing statement
    closing_text = "That's all for this section. Thank you for using me. Have a nice day"
    print("\n🔊 Generating closing speech...")
    closing_audio_path = tts_engine.generate_speech(
        text=closing_text,
        emotion='neutral', # Neutral emotion for closing statement
        intensity=1.0
    )
    closing_audio = AudioSegment.from_mp3(closing_audio_path)
    combined_narration_audio += closing_audio

    # Define base audio for mixing
    base_audio = combined_narration_audio

    # Handle sound effects based on toggle
    if sound_toggle.value == 'With Effects':
        print("🎵 Playing with sound effects (spaced out)...")
        final_audio_segment = base_audio

        # Overlay remaining effects at spaced intervals
        effect_interval = len(base_audio) / (len(current_effects_list) + 1) if current_effects_list else 0
        current_position = effect_interval

        for effect_name in current_effects_list:
            if effect_name in sound_mixer.sound_effects:
                effect = sound_mixer.sound_effects[effect_name]
                # Adjust volume to be subtle (-15dB to -20dB)
                effect = effect - 15

                # Overlay once at current_position
                if current_position + len(effect) < len(final_audio_segment):
                    final_audio_segment = final_audio_segment.overlay(effect, position=current_position)
                current_position += effect_interval # Move to next position

        final_audio_path = '/content/mixed_lesson.mp3'
        final_audio_segment.export(final_audio_path, format='mp3')
        final_audio = final_audio_path

    elif sound_toggle.value == 'Only Effects':
        print("🎵 Playing only sound effects (spaced out)...")
        # Create a silent background with the same duration as the combined narration
        silent_background = AudioSegment.silent(duration=len(base_audio))
        final_audio_segment = silent_background

        # Overlay remaining effects at spaced intervals
        effect_interval = len(silent_background) / (len(current_effects_list) + 1) if current_effects_list else 0
        current_position = effect_interval

        for effect_name in current_effects_list:
            if effect_name in sound_mixer.sound_effects:
                effect = sound_mixer.sound_effects[effect_name]
                # Adjust volume to be subtle (-15dB to -20dB)
                effect = effect - 15

                # Overlay once at current_position
                if current_position + len(effect) < len(final_audio_segment):
                    final_audio_segment = final_audio_segment.overlay(effect, position=current_position)
                current_position += effect_interval # Move to next position

        final_audio_path = '/content/only_effects_lesson.mp3'
        final_audio_segment.export(final_audio_path, format='mp3')
        final_audio = final_audio_path

    else:
        # Narration only
        combined_narration_path = '/content/combined_narration.mp3'
        base_audio.export(combined_narration_path, format='mp3')
        final_audio = combined_narration_path
        print("🔊 Playing narration only...")


    # Play the audio
    display(Audio(final_audio, autoplay=True))

    # Display text (optional)
    print("\n📖 Lesson Text Preview:")
    print(text_to_speak[:300] + "...")

# Connect chapter selection to lesson updates
chapter_dropdown.observe(update_lessons, names='value')

# Attach the observer to the grade_selector widget
grade_selector.observe(on_grade_change, names='value')

# Connect play button
play_button.on_click(play_selected_lesson)

# Trigger the initial load and update based on the default selected grade
on_grade_change({'new': grade_selector.value})

Loading data for Grade 10...
